## PPI General Linear Model

Here, PPI network construction is performed by evaluating multiple linear models and extracting regressors beta weights. PPI linear model takes form: 

$$y=\beta_0
+\beta_1\cdot x_{physio}
+\beta_2\cdot x_{out\_ons}
+\beta_3\cdot x_{out\_perr}
+\beta_4\cdot x_{PPI:out\_ons}
+\beta_5\cdot x_{PPI:out\_perr}
+\beta_6\cdot x_{dec\_ons}
+\beta_7\cdot x_{dec\_miss}
+\beta_8\cdot x_{dec\_wcor}
+\beta_9\cdot x_{res\_lbp}
+\beta_{10}\cdot x_{res\_rbp}
+\beta_{11}\cdot x_{res\_miss}
+\beta_{12}\cdot x_{out\_off}$$

Detailed regressors description:

| Regressor | Type | Description | Event Duration
|:--:|----|----|----|
| `y` | physiological | BOLD signal from target region | – |
| | | | |
| `physio` | physiological | BOLD signal from seed region | – |
| `out_ons` | psychological | outcome phase onset | `t_event_psycho` |
| `out_perr` | psychological | outcome phase onset parametrically modulated with prediction error | `t_event_psycho` |
| `ppi_out_ons` | interaction | `out_ons` point-by-point multiplied with <br />deconvolved seed timeseries, reconvolved with HRF | `t_event_ppi` 
| **`ppi_out_perr`** | interaction | `out_perr` point-by-point multiplied with <br />deconvolved seed timeseries, reconvolved with HRF | `t_event_ppi` |
| `dec_ons` | psychological | decision phase onset | `t_event_psycho` |
| `dec_miss` | psychological | decision phase onset for trials with missing response | `t_event_psycho` |
| `dec_wcor` | psychological | decision phase onset parametrically modulated with expected probability of being correct | `t_event_psycho` |
| `res_lbp` | psychological | left button press onset | `t_event_psycho` |
| `res_rbp` | psychological | right button press onset | `t_event_psycho` |
| `res_miss` | psychological | onset of isi phase for trials with missing response | `t_event_psycho` |
| `out_off` | psychological | outcome phase offset | `t_event_psycho` |
| `intercept` | other | intercept for linear model | – |

Each type of regressor was calculated differently: 
- **physiological regressors**: BOLD signals extracted using `NiftiSpheresMasker`. Before extraction confounds (24 head motion parameters, CSF and WM signals, squares, temporal derivatives and squares of temporal derivatives) and low-frequency drift were removed and signal was high pass-filtered (128s), 
- **psychological regressors**: task events convolved with canonical HRF using `compute_regressor` function from `nistats.hemodynamic_models` wrapped in custom `Regressor` class for consistent treating standard and parametrically modulated regressor, 
- **interaction (PPI) regressors**: first, ROIs timeseries were extracted with detrending and high-pass filtering. Then they were deconvolved using spm fuction for parameter estimation Bayes `spm_PEB.m`. Deconvolved and upsampled (by default 16 times) ROI timeseries were point-by-point multiplied with upsampled and demeaned task events timeseries to create interaction regressor in neural space. Finally, these interaction regressors were reconvolved with HRF and downsampled to create PPI regressors in BOLD space.

---
**Last update**: 31.08.2020

In [1]:
import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import statsmodels.regression.linear_model as sm
from bids import BIDSLayout
from IPython.display import clear_output
from scipy import io
from scipy.stats import zscore

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)
from dn_utils.behavioral_models import load_behavioral_data
from dn_utils.glm_utils import (convolve, Regressor, upsampled_events,
                                my_make_first_level_design_matrix)
from dn_utils.plotting import plot_design_matrix, plot_regressors_correlation

%matplotlib inline

/home/kmb/Desktop/Neuroscience/Projects/BONNA_decide_net/code/dn_utils/glm_utils.py:13: FutureWarning: 

 | Starting with Nilearn 0.7.0, all Nistats functionality has been incorporated into Nilearn's stats & reporting modules.
 | Nistats package will no longer be updated or maintained.

  from nistats import design_matrix


In [2]:
# Directory for PPI analysis
path_nistats = os.path.join(path_root, 'data/main_fmri_study/derivatives/nistats')
path_out = os.path.join(path_root, 'data/main_fmri_study/derivatives/ppi')
path_betamats = os.path.join(path_out, 'betamats')
path_timeries = os.path.join(path_out, 'timeseries')
path_parcellations = os.path.join(path_out, 'parcellations')

# Load behavioral data
path_beh = os.path.join(path_root, 'data/main_fmri_study/sourcedata/behavioral')
beh, meta = load_behavioral_data(path=path_beh, verbose=False)
n_subjects, n_conditions, n_trials, _ = beh.shape

# Load trial modulations
path_modulations = os.path.join(path_nistats, 'modulations')
modulations_wcor = np.load(os.path.join(path_modulations, 
                                        'modulations_wcor.npy'))
modulations_perr = np.load(os.path.join(path_modulations, 
                                        'modulations_perr.npy'))

# Load neural & BOLD timeseries
data = io.loadmat(os.path.join(
    path_timeries, 
    'timeseries_pipeline-24HMPCSFWM_atlas-customROI_neural.mat'))
timeseries_neural_aggregated = data['timeseries_neural_aggregated']
timeseries_denoised_aggregated = np.load(os.path.join(
    path_timeries, 
    'timeseries_pipeline-24HMPCSFWM_atlas-customROI_bold.npy'))
downsamples = data['k'].flatten()

# Load region labels
with open(os.path.join(path_parcellations, 
                       'custom_roi/custom_roi_labels.txt'), 'r') as f:
    roi_labels = f.read().splitlines() 
roi_labels = [label.replace('_', '')[:-1] + label[-1].lower() 
              for label in roi_labels]
    
# Create directory for output
atlas_name = 'customROI'
path_results = os.path.join(path_betamats, atlas_name)
Path(path_results).mkdir(exist_ok=True)

# Acquisition parameters
_, _, n_volumes, n_rois = timeseries_denoised_aggregated.shape
t_r = 2
frame_times = np.arange(n_volumes) * t_r

# Duration of phases
t_dec, t_out = 1.5, 1.5

# Upsampling rate for deconvolved signal
sampling_rate = 1/16

# Input data shape
print('beh.shape', beh.shape)
print('modulations_wcor.shape', modulations_wcor.shape)
print('modulations_perr.shape', modulations_perr.shape)
print('timeseries_neural_aggregated.shape', timeseries_neural_aggregated.shape)
print('timeseries_denoised_aggregated.shape', timeseries_denoised_aggregated.shape)
print('downsamples.shape', downsamples.shape)

beh.shape (32, 2, 110, 23)
modulations_wcor.shape (32, 2, 110)
modulations_perr.shape (32, 2, 110)
timeseries_neural_aggregated.shape (32, 2, 11680, 14)
timeseries_denoised_aggregated.shape (32, 2, 730, 14)
downsamples.shape (730,)


### PPI GLM events duration
- `t_event_psycho`: duration of all psychological events (e.g. button press)
- `t_event_ppi`: duration of psychological events for upsampled regressors
---

In [3]:
t_event_psycho = 1
t_event_ppi = 1

# Create paths to store output data
path_save = os.path.join(
    path_results, 
    f'tpsycho-{int(t_event_psycho * 1000)}_tppi-{int(t_event_ppi * 1000)}')
path_save_dm = os.path.join(
    path_save, 'design_matrices'
)
path_save_reg_corr = os.path.join(
    path_save, 'regressors_correlation'
)

Path(path_save_dm).mkdir(exist_ok=True, parents=True)
Path(path_save_reg_corr).mkdir(exist_ok=True, parents=True)

---

#### Create design matrix

> Note that for different subject / task entities design matrix may have or not have two additional regressors (miss for decision onset and miss for decision offset). Some subjects didn't miss any response.

In [4]:
# Relevant regressors to save (regressors that all models have)
beta_names = ['physio', 
              'out_ons', 'out_perr', 
              'ppi_out_ons', 'ppi_out_perr', 
              'dec_ons', 'dec_wcor', 
              'res_lbp', 'res_rbp',
              'out_off', 
              'reg_intercept']

# Keys are all regressors (even miss regressors which some of participants may 
# not have)
beta_colors = {
    'physio': 'tab:red', 
    'out_ons': 'tab:blue',
    'out_perr': 'tab:purple',
    'ppi_out_ons': 'tab:green',
    'ppi_out_perr': 'tab:orange',
    'dec_ons': 'tab:blue',
    'dec_miss': 'tab:blue',
    'dec_wcor':'tab:purple',
    'res_lbp': 'tab:blue',
    'res_rbp': 'tab:blue',
    'res_miss': 'tab:blue',
    'out_off': 'tab:blue',
    'reg_intercept': 'tab:gray'
}

n_regressors = len(beta_names)  

In [5]:
for con, con_name in enumerate(meta['dim2']):
    for sub, sub_name in enumerate(meta['dim1']):

        beta_mats = np.zeros((n_regressors, n_rois, n_rois))
        #pval_mats = np.zeros((n_regressors, n_rois, n_rois))
        #tval_mats = np.zeros((n_regressors, n_rois, n_rois))
        
        # Extract task events
        resp_type = beh[sub, con, :, meta['dim4'].index('response')]
        onset_out = beh[sub, con, :, meta['dim4'].index('onset_out')]
        onset_dec = beh[sub, con, :, meta['dim4'].index('onset_dec')] 
        onset_res = beh[sub, con, :, meta['dim4'].index('onset_dec')] + \
                    beh[sub, con, :, meta['dim4'].index('rt')]
        offset_dec = onset_dec + t_dec
        offset_out = onset_out + t_out

        modulation_wcor = modulations_wcor[sub, con, resp_type != 0]
        modulation_perr = modulations_perr[sub, con]
        modulation_wcor_demeaned = modulation_wcor - np.mean(modulation_wcor)
        modulation_perr_demeaned = modulation_perr - np.mean(modulation_perr)

        for idx_seed in range(n_rois):

            # Physiological regressor (seed time-series)
            reg_physio = Regressor.from_values(
                'physio', 
                frame_times, 
                timeseries_denoised_aggregated[sub, con, :, idx_seed])            

            # Psychological regressors 
            reg_out_perr = Regressor(
                name='out_perr', 
                frame_times=frame_times,
                duration=np.ones(n_trials) * t_event_psycho, 
                onset=onset_out,
                modulation=modulation_perr_demeaned)
            reg_out_ons = Regressor(
                name='out_ons', 
                frame_times=frame_times,
                duration=np.ones(n_trials) * t_event_psycho, 
                onset=onset_out)

            # PPI regressors
            ts_neural_up = timeseries_neural_aggregated[sub, con, :, idx_seed]

            ts_out_perr_up = upsampled_events(
                t_r=t_r,
                n_volumes=n_volumes,
                onset=onset_out,
                duration=t_event_ppi, 
                modulation=modulation_perr_demeaned)
            ts_out_ons_up = upsampled_events(
                t_r=t_r,
                n_volumes=n_volumes,
                onset=onset_out,
                duration=t_event_ppi)

            # Point by point multiplication
            ts_ppi_out_perr_up = ts_neural_up * ts_out_perr_up
            ts_ppi_out_ons_up = ts_neural_up * ts_out_ons_up

            # Reconvolution
            reg_ppi_out_perr = Regressor.from_values(
                'ppi_out_perr',
                frame_times,
                values=convolve(ts_ppi_out_perr_up, t_r=t_r*sampling_rate)[downsamples]
            )
            reg_ppi_out_ons = Regressor.from_values(
                'ppi_out_ons',
                frame_times,
                values=convolve(ts_ppi_out_ons_up, t_r=t_r*sampling_rate)[downsamples]
            )

            # No-interest regressors
            reg_dec_ons = Regressor(
                'dec_ons', 
                frame_times, 
                onset_dec[resp_type != 0],
                duration=np.ones(len(onset_dec[resp_type != 0])) * t_event_psycho)
            reg_dec_miss = Regressor(
                'dec_miss', 
                frame_times, 
                onset_dec[resp_type == 0],
                duration=np.ones(len(onset_dec[resp_type == 0])) * t_event_psycho)
            reg_dec_wcor = Regressor(
                'dec_wcor', 
                frame_times, 
                onset_dec[resp_type != 0],
                duration=np.ones(len(onset_dec[resp_type != 0])) * t_event_psycho,
                modulation=modulation_wcor_demeaned)
            reg_res_lbp = Regressor(
                'res_lbp', 
                frame_times, 
                onset_res[resp_type == -1],
                duration=np.ones(len(onset_res[resp_type == -1])) * t_event_psycho)
            reg_res_rbp = Regressor(
                'res_rbp', 
                frame_times, 
                onset_res[resp_type == 1],
                duration=np.ones(len(onset_res[resp_type == 1])) * t_event_psycho)
            reg_res_miss = Regressor(
                'res_miss', 
                frame_times, 
                offset_dec[resp_type == 0],            
                duration=np.ones(len(offset_dec[resp_type == 0])) * t_event_psycho)
            reg_out_off = Regressor(
                'out_off', 
                frame_times, 
                offset_out,
                duration=np.ones(len(offset_out)) * t_event_psycho)

            # Create design matrix
            regressors = [
                reg_physio,                              # physiological
                reg_out_ons, reg_out_perr,               # main psychological
                reg_ppi_out_ons, reg_ppi_out_perr,       # PPI
                reg_dec_ons, reg_dec_miss, reg_dec_wcor, # no-interest (dec)
                reg_res_lbp, reg_res_rbp, reg_res_miss,  # no-interest (res)
                reg_out_off                              # no-interest (out)
            ] 

            # Create design matrix
            X, _ = my_make_first_level_design_matrix(regressors)
            X = X[[c for c in X.columns if 'drift' not in c]]
            X = X.drop('constant', axis=1)
            Xstd = (X - X.mean()) / X.std()
            Xstd['reg_intercept'] = np.ones(n_volumes)
            
            # Save design matrix plot and regressor correlation plot
            dm_plot_fname = os.path.join(
                path_save_dm, 
                f'sub-{sub_name}_task-{con_name}_seed-{roi_labels[idx_seed]}' \
                + '_dmplot.png'
            )
            reg_corr_plot_fname = os.path.join(
                path_save_reg_corr, 
                f'sub-{sub_name}_task-{con_name}_seed-{roi_labels[idx_seed]}' \
                + '_regcorrplot.png'
            )
            plot_design_matrix(
                Xstd, 
                colors=[beta_colors[c] for c in Xstd.columns],
                output_file=dm_plot_fname
            )
            plot_regressors_correlation(
                Xstd, 
                colors=[beta_colors[c] for c in Xstd.columns], 
                output_file=reg_corr_plot_fname
            )
            
            for idx_target in range(n_rois):        
                
                # Modeled response (target time-series)
                y = pd.DataFrame(
                    timeseries_denoised_aggregated[sub, con, :, idx_target],
                    columns=['target'], 
                    index=frame_times)

                # Fit GLM
                model = sm.OLS(y, Xstd, hasconst=True)
                results = model.fit()
                
                beta_mats[:, idx_target, idx_seed] = results.params.loc[beta_names]
                #pval_mats[:, idx_target, idx_seed] = results.pvalues.loc[beta_names]
                #tval_mats[:, idx_target, idx_seed] = results.tvalues.loc[beta_names]
                
                
        # Save beta estimates
        print(f'saving sub-{sub_name}_task-{con_name}_betamats.npy')
        fname = os.path.join(
            path_save, 
            f'sub-{sub_name}_task-{con_name}_betamats.npy')
        np.save(fname, beta_mats)

saving sub-m02_task-rew_betamats.npy
saving sub-m03_task-rew_betamats.npy
saving sub-m04_task-rew_betamats.npy
saving sub-m05_task-rew_betamats.npy
saving sub-m06_task-rew_betamats.npy
saving sub-m07_task-rew_betamats.npy
saving sub-m08_task-rew_betamats.npy
saving sub-m09_task-rew_betamats.npy
saving sub-m10_task-rew_betamats.npy
saving sub-m11_task-rew_betamats.npy
saving sub-m12_task-rew_betamats.npy
saving sub-m13_task-rew_betamats.npy
saving sub-m14_task-rew_betamats.npy
saving sub-m15_task-rew_betamats.npy
saving sub-m16_task-rew_betamats.npy
saving sub-m17_task-rew_betamats.npy
saving sub-m18_task-rew_betamats.npy
saving sub-m19_task-rew_betamats.npy
saving sub-m20_task-rew_betamats.npy
saving sub-m21_task-rew_betamats.npy
saving sub-m22_task-rew_betamats.npy
saving sub-m23_task-rew_betamats.npy
saving sub-m24_task-rew_betamats.npy
saving sub-m25_task-rew_betamats.npy
saving sub-m26_task-rew_betamats.npy
saving sub-m27_task-rew_betamats.npy
saving sub-m28_task-rew_betamats.npy
s